# S&P 500 Data Project

## Project overview
The following project aims to get data on all stocks in the S&P500 index by using the Alpha Vantage API. Following this, we choose some big-tech company stocks and present their prices in an interactive figure. Lastly, we identify stocks exhibiting short-term momentum.

## **Note :** Please don't run the cell that calls the API, since it will run for about 2 hours. We have completed that step locally and saved the resulting dataset locally.

## Imports and set magics:

In [37]:
# a. Import

import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries # Enables the use of Alpha Vantage stock API
import time
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import plotly.graph_objects as go
import requests

# b. Autoreload

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
# import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Call, read, clean and present data

## Getting all S&P500 tickers

**Read all S&P500 tickers** from ``http://en.wikipedia.org/wiki/List_of_S%26P_500_companies`` and **save it** as variable:

In [ ]:
# a. Read

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

# b. Save

tickers = save_sp500_tickers()
tickers = [x[:-1] for x in tickers]

# c. Replacing ticker names that later produces errors:

tickers = [w.replace('BF.B', 'BF-B') for w in tickers]

The first 5 ticker namesnow looks like this:

In [ ]:
print(tickers[0:5])

## Call API to get all S&P500 data

**Call Alpha Vantage API to get all S&P500 monthly stock prices** and **save it** as dataset:

In [ ]:
api_key = 'IHXRIDZAXOXUHF8T' # PERSONAL AlphaVantage API key. Please, don't abuse/redistribute.

ETFs = np.array(tickers) # All S&P 500 tickers used.

ts = TimeSeries(key = api_key, output_format = 'pandas', indexing_type='date') # From alpha_vantage.

**DON'T RUN THE FOLLOWING CELL**:

In [ ]:
price_data = pd.DataFrame()

# Loop through requested stocks.

print('Querying Securities, Estimated time: ' + str(round(len(ETFs)/5)) + ' minutes') # Message for estimated time left.
for x in range(len(ETFs)):
    print(str(ETFs[x]))

    if (x + 1) % 5 == 0:
        time.sleep(60) # The free version of the API is limited to 5 calls per minute.

    data, meta_data = ts.get_monthly_adjusted(symbol=str(ETFs[x])) 
    data = data['5. adjusted close'].iloc[::-1]             
    data = pd.DataFrame(data).rename(index=str, columns={'5. adjusted close' : str(ETFs[x])})
    price_data = pd.concat([price_data,data], axis=1, sort=False)
    
price_data = price_data.iloc[:300].iloc[::-1]

plt.plot(price_data)
plt.xticks([])

Create **local copy** for future references so the API that takes hours to run won't have to be called more than once:

In [4]:
localdata = price_data.copy()

localdata.to_csv('localdata.csv')

NameError: name 'price_data' is not defined

# Start here!

**Loading dataset from local path**:

In [15]:
df = pd.read_csv('localdata.csv', delimiter = ",", index_col = 'Unnamed: 0')
df = df.reindex(index = df.index[::-1]) # Changing the order of rows.
df = df.iloc[:-1, :]
print(df.tail())

MMM      ABT     ABBV    ABMD       ACN    ATVI  \
2019-10-31 00:00:00  162.0725  83.2604  78.4998  207.58  184.7071  56.030   
2019-11-29 00:00:00  168.2221  85.0927  86.5718  196.18  200.3865  54.830   
2019-12-31 00:00:00  174.8115  86.4968  87.3711  170.59  209.7603  59.420   
2020-01-31 00:00:00  157.2134  87.1400  81.0200  186.29  205.2100  58.480   
2020-02-21 00:00:00  156.9300  87.4500  94.9600  166.74  211.8900  63.375   

                       ADBE    AMD       AAP      AES  ...      WYNN      XEL  \
2019-10-31 00:00:00  277.93  33.93  162.4177  16.9298  ...  120.3275  63.1034   
2019-11-29 00:00:00  309.53  39.15  157.0198  18.7767  ...  120.8500  61.0964   
2019-12-31 00:00:00  329.81  45.86  160.1600  19.7597  ...  138.8700  63.4900   
2020-01-31 00:00:00  351.14  47.00  131.7500  19.8600  ...  126.1600  69.1900   
2020-02-21 00:00:00  372.95  53.28  140.6100  20.7500  ...  127.9500  70.8200   

                         XRX     XLNX    XYL       YUM    ZBRA       ZBH  \


**Subsetting** a couple of big tech company stocks to focus on in the analysis:

In [32]:
df_big = np.log(df[['MSFT', 'AMZN', 'AAPL', 'GOOG', 'FB']]) # Choosing certain big tech stocks for the interactive figures.

df_big['Date'] = df.index

df_big['Date'] = pd.to_datetime(df_big.Date)

df_big['Date'] = df_big['Date'].dt.strftime('%m/%Y')

df_big = df_big.drop(df.index[0 : 169]) # Deleting rows to enhance the graph and only show returns from past couple of years.

print(df_big.head())

MSFT      AMZN      AAPL      GOOG        FB     Date
2014-04-30 00:00:00  3.568904  5.717455  4.332368  6.266555  4.090671  04/2014
2014-05-30 00:00:00  3.589084  5.744764  4.408143  6.327740  4.147885  05/2014
2014-06-30 00:00:00  3.607480  5.783148  4.435430  6.354857  4.209012  06/2014
2014-07-31 00:00:00  3.641893  5.746171  4.463755  6.348439  4.285653  07/2014
2014-08-29 00:00:00  3.699307  5.826118  4.538408  6.348439  4.315085  08/2014


## Present stock data

**Creating** an **interactive figure** to show the development of stock prices:

In [34]:
# a. Initialize figure

fig = go.Figure()

# b. Add scatters for each big tech ticker

fig.add_trace(
    go.Scatter(x = list(df_big.Date), y = list(df_big.MSFT) , name = 'Microsoft'))

fig.add_trace(
    go.Scatter(x = list(df_big.Date), y = list(df_big.AMZN) , name = 'Amazon'))

fig.add_trace(
    go.Scatter(x = list(df_big.Date), y = list(df_big.AAPL) , name = 'Apple'))

fig.add_trace(
    go.Scatter(x = list(df_big.Date), y = list(df_big.FB) , name ='Facebook'))

fig.add_trace(
    go.Scatter(x = list(df_big.Date), y = list(df_big.GOOG) , name = 'Google'))

# c. Add slicer

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="None",
                     method="update",
                     args=[{"visible": [True, True, True, True, True]}]),
                dict(label="Microsoft",
                     method="update",
                     args=[{"visible": [True, False, False, False, False]}]),
                dict(label="Amazon",
                     method="update",
                     args=[{"visible": [False, True, False, False, False]}]),
                dict(label="Apple",
                     method="update",
                     args=[{"visible": [False, False, True, False, False]}]),
                dict(label="Facebook",
                     method="update",
                     args=[{"visible": [False, False, False, True, False]}]),
                dict(label="Google",
                     method="update",
                     args=[{"visible": [False, False, False, False, True]}])
            ]),
        )
    ])

# d. Add creative title and display

fig.update_layout(title_text="Stocks" , yaxis_title = 'Log - Adj. Close')


fig.show()

# Analyze the data for momentum signals

## Calculating returns and identifying momentum stocks

In [18]:
returns_df = df.copy() # Copy of dataset, to make return calculations

returns_df = returns_df.apply(lambda x: x.shift(-1)/x - 1, axis = 0) # Monthly returns 

nreturns_df = returns_df.shift(1, axis = 0) # Offsets the rows by 1, to make the results more intuitive

**Accumulating returns to define winners and losers**:

In [19]:
creturns_df = nreturns_df.copy()

creturns_df = creturns_df.apply(lambda x : x + x.shift(1) + x.shift(2) + x.shift(3) + x.shift(4) + x.shift(5), axis = 0) 
# Accumulating returns for the previous 6 months

creturns_df = creturns_df.shift(1, axis = 0) #Offsets values

print(creturns_df.iloc[0:10,:])

MMM       ABT  ABBV      ABMD  ACN      ATVI  \
2000-03-31 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-04-28 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-05-31 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-06-30 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-07-31 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-08-31 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-09-29 00:00:00       NaN       NaN   NaN       NaN  NaN       NaN   
2000-10-31 00:00:00  0.046710  0.322759   NaN  0.689462  NaN  0.576326   
2000-11-30 00:00:00  0.128859  0.340331   NaN  0.623986  NaN  0.978843   
2000-12-29 00:00:00  0.162977  0.324406   NaN  0.404348  NaN  0.739336   

                         ADBE       AMD  AAP       AES  ...  WYNN       XEL  \
2000-03-31 00:00:00       NaN       NaN  NaN       NaN  ...   NaN       NaN   
2000-04-28 00:00:00       NaN       NaN  NaN       Na

**Defining winners and losers based on distribution**

Stocks in the top 30 percentiles, have strong positive momentum signals, and should therefore be bought.

Likewise stocks in the bottom 30 percentiles have strong negative momentum signals and should therefore be shorted.

In [20]:
dreturns_df = creturns_df.copy()

dreturns_df = dreturns_df.transpose() # Transposes the dataset, so it's applicable for .describe()

perc = [0.3, 0.7] # Defining lower and upper bounds

dreturns_df.describe(percentiles = perc)

,2000-03-31 00:00:00,2000-04-28 00:00:00,2000-05-31 00:00:00,2000-06-30 00:00:00,2000-07-31 00:00:00,2000-08-31 00:00:00,2000-09-29 00:00:00,2000-10-31 00:00:00,2000-11-30 00:00:00,2000-12-29 00:00:00,...,2019-05-31 00:00:00,2019-06-28 00:00:00,2019-07-31 00:00:00,2019-08-30 00:00:00,2019-09-30 00:00:00,2019-10-31 00:00:00,2019-11-29 00:00:00,2019-12-31 00:00:00,2020-01-31 00:00:00,2020-02-21 00:00:00
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,376.000000,377.000000,378.000000,...,497.000000,498.000000,498.000000,498.000000,498.000000,498.000000,501.000000,501.000000,502.000000,504.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.129447,0.149894,0.108105,...,0.117960,0.021199,0.197979,0.108329,0.040598,0.062773,0.039121,0.141329,0.091813,0.064118
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.355046,0.354822,0.341658,...,0.147569,0.160996,0.151693,0.147333,0.179743,0.145652,0.143189,0.137143,0.123675,0.132420
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.400614,-1.707232,-1.875295,...,-0.503702,-0.567876,-0.324725,-0.362056,-0.503271,-0.588663,-0.672935,-0.402649,-0.334614,-0.434771
30%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016224,0.010782,-0.039163,...,0.057641,-0.040999,0.142191,0.043323,-0.042677,0.006235,-0.012148,0.079387,0.040607,-0.001145
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136579,0.142811,0.116519,...,0.134049,0.039867,0.200928,0.122050,0.060126,0.083149,0.053398,0.138991,0.089289,0.065076
70%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277772,0.287931,0.240920,...,0.182794,0.107436,0.270701,0.181353,0.149655,0.144108,0.117993,0.204464,0.147422,0.131387
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.707187,1.872311,1.906224,...,0.530839,0.529865,0.839585,0.555182,0.540045,0.448234,0.369864,0.581010,0.607600,0.475121


In [22]:
# Creating list of percentiles on different dates

percentiles = pd.DataFrame()

percentiles['lower bound'] = dreturns_df.quantile(0.3)
percentiles['upper bound'] = dreturns_df.quantile(0.7)

print(percentiles.iloc[[-4], [0,1]]) #

# Adding bounds to dataframe , to quickly compare whether stocks are winners or losers

creturns_df['lower bound'] = percentiles['lower bound']
creturns_df['upper bound'] = percentiles['upper bound']

print(creturns_df.tail())

lower bound  upper bound
2019-11-29 00:00:00    -0.012148     0.117993
                          MMM       ABT      ABBV      ABMD       ACN  \
2019-10-31 00:00:00 -0.193951  0.060752 -0.018948 -0.406095  0.100710   
2019-11-29 00:00:00 -0.102433  0.064721  0.048742 -0.210552  0.022476   
2019-12-31 00:00:00  0.085229  0.129841  0.185327 -0.209632  0.132546   
2020-01-31 00:00:00  0.039330  0.041652  0.246575 -0.334614  0.141699   
2020-02-21 00:00:00 -0.069301  0.009467  0.243820 -0.311950  0.077738   

                         ATVI      ADBE       AMD       AAP       AES  ...  \
2019-10-31 00:00:00  0.163417  0.046877  0.139963 -0.005399 -0.073033  ...   
2019-11-29 00:00:00  0.163322 -0.032448  0.227687  0.001634  0.024324  ...   
2019-12-31 00:00:00  0.242299  0.144690  0.389495  0.036460  0.210518  ...   
2020-01-31 00:00:00  0.237703  0.122538  0.452897  0.061619  0.202113  ...   
2020-02-21 00:00:00  0.189256  0.172923  0.475121 -0.093060  0.197254  ...   

                     

## Comparing stock performance and bounds to see if they exhibit momentum

In [27]:
# Compares if the Big-Tech stocks are "Winners" or "Losers"

print(creturns_df.loc['2019-11-29 00:00:00' , ['MSFT', 'AMZN', 'AAPL', 'GOOG', 'FB' , 'lower bound' , 'upper bound']])

MSFT           0.105676
AMZN          -0.073733
AAPL           0.249963
GOOG           0.069869
FB             0.002764
lower bound   -0.012148
upper bound    0.117993
Name: 2019-11-29 00:00:00, dtype: float64


**Results:**

It is **evident** that neither **Microsoft**, **Google** nor **Facebook** exhibit momentum, granted they fall *within* the bounds.

**However**, the **Apple** stock exhibits **positive short-term momentum** (*long signal*), whilst the **Amazon** stock exhibits **negative short-term momentum** (*short signal*).

# Conclusion

We have shown how easily stock price data may be called from an API and how it is easily vizualised. Furthermore, we have analyzed the data and offered a trading recommendation based on a simple version of a momentum strategy (Evaluate 6 months and hold stocks for 6 months).